In [1]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Chama as classes do databases
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
rel_off = off_Reliance('Primeny')


In [3]:
##Informações necessárias para rodar a otimização das carteiras- gerar as movimentações

#Tabela com as informações das SC de gestão offshore (nomes, responsáveis e perfil de cada cliente)
po_cadastro = bawm.po_cadastro_all().drop(columns=['CodigoProduto'])

#Puxa da PO_Cadastro os responsaveis por cada supercarteira de gestão (DPMS)

#Lista com os usuários e emails de toda Julius Baer Brazil.
emails_usuarios = crm.lista_usuarios_u()

#Verificar os responsaveis por cada sc de gestão com base no mapa de ativos
responsaveis = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore\Mapa de Ativos/Mapa JBFO_v2.xlsx',sheet_name='Sheet1')
responsaveis = responsaveis[(responsaveis['Origem']=='Offshore')& (responsaveis['Tipo de Contrato']!='Distrato em Andamento')][['Conta','SC/ContaMov','RM','Co-RM','Controller','Controller Backup','Segmentação']]

#Altera os usuários u para letra minúscula para padronização (será usado no envio de e-mail)
responsaveis['RM'] =responsaveis['RM'].str.lower()
responsaveis['Co-RM'] =responsaveis['Co-RM'].str.lower()
responsaveis['Controller'] = responsaveis['Controller'].str.lower()
responsaveis['Controller Backup'] = responsaveis['Controller Backup'].str.lower()
#responsaveis['DeputyDPMUsuarioU'] = responsaveis['DeputyDPMUsuarioU'].str.lower()


In [4]:
# Última data do mes anterior para rodar a query do extrato
data = bds.banco.hoje()
data = datetime.datetime.date(data)

#Traz a alocação ideal por perfil de portfolio
alocacao_por_sc = crm.alocacao_ideial_off()

#Traz os fundos da JBFO (prioridade na aplicação das movimentações)
fundos_jbfo = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')
fundos_jbfo = fundos_jbfo[['NomeDoProduto','CodigoProduto','Classificação']]
fundos_jbfo['Fundo']=fundos_jbfo['NomeDoProduto'].str.lower()

#de_paras de mandatos (PI anterior para a atual - equivalência)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
#Traz os deparas de classificação gps para classes de asset Allocation.
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos - Pontos Táticos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Clientes que estão em distrato
distrato = crm.clientes_em_distrato_off()

#Trazer os produtos GPS e segregar o que é imobiliario (caso precise segregar o que é Reits / ILiquidos)
produtos_gps = crm.produtos_off()
produtos_gps =produtos_gps[['name','new_idmysql','new_tickerid','new_isin','new_idbds','new_idsistemaorigem','new_indgroupaxysidname','transactioncurrencyidname']].rename(columns={'name':'Ativo','new_idmysql':'IdMysqlProduto','new_tickerid':'Ticker','new_isin':'ISIN','new_idsistemaorigem':'CodigoProduto','transactioncurrencyidname':'Moeda'}).reset_index()
produtos_gps['ISIN'] = produtos_gps['ISIN'].str.lower()

#Retorna o nome dos veiculos que fazemos gestao (substituindo a classe JBFO)
dicio_veiculos = dict(pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')[['Classe','Veiculo Abreviado']].drop_duplicates().values)



In [5]:
#Funções usadas na otimização de carteiras

#Retira espaços inúteis
def arruma_espaco(x):
    while x[-1]==' ':
        x=x[:-1]
    return x    
    
#Formatar valor para envio de e-mail
def formatar(valor):
    int_part, dec_part = format(valor, ',.2f').split('.')
    return f"{int_part.replace(',', '.')},{dec_part}"

#Trazer pu do BDS (ainda não usado)
def trazer_pu_atual(id_serie):
    try:
        df = bds.serie_historico(idserie=id_serie, intervalo=1)
        if df.empty:
            df = bds.serie_historico(idserie=id_serie, intervalo=4)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)          
        else:
            df = bds.serie_historico(idserie=id_serie, intervalo=1)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)
    except:
        cota=np.nan
        ultima_data = np.nan
    return cota, ultima_data    

#Função que identifica clientes inativos
def inativo(x):
    x = x.replace('(','').replace(')','')
    x = x.replace(' ','')
    x = x.strip()
    if 'inativ' in x.lower():
        valor='inativo'
    else:
        valor='ok'
    return valor   

In [6]:
#Consolida a posição atual versus a ideial
alocacao_por_sc = pd.pivot_table(data=alocacao_por_sc, values = 'new_percentual',columns=['new_classedeinvestimentoidname'],index=['new_name','new_conta_supercarteiraidname','new_politicadeinvestimentoidname']).reset_index()
alocacao_por_sc['new_name']=alocacao_por_sc['new_name'].str.lower()
alocacao_por_sc = pd.merge(left = alocacao_por_sc, right = po_cadastro,left_on ='new_conta_supercarteiraidname',right_on = 'NomeSuperCarteiraCRM',how='left')
alocacao_por_sc['inativo']=alocacao_por_sc['new_name'].apply(lambda x: inativo(x))
alocacao_por_sc=alocacao_por_sc[alocacao_por_sc['inativo']!='inativo']
alocacao_por_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname','Alt Crédito', 'Alt Multistr','Alt RV L/S', 'Alt Trad/Macro', 'Alt Valor Rel', 'Commodities','Imobiliário', 'Outros', 'P. Equity', 'RF CP', 'RF Glob', 'RF Infl','RF Livre -', 'RF Outros -', 'RV', 'RV Outros -','new_politicadeinvestimentoidname','Titularidade','VetoPowerGestaoTatica','DPM','OfficerUsuarioU','ControllerUsuarioU']].reset_index(drop=True)

In [7]:
##Traz a posição do último extrato gerado

#Retorna o último dia do mes
ultimo_extrato = data - relativedelta(days=data.day -1)
ultimo_extrato = ultimo_extrato + timedelta(days = -1)

#Gera a carteira da primeira sc da lista
teste = alocacao_por_sc.drop_duplicates()
sc = teste.iloc[0,0]
sc = '@5AGL1'
df = extrato.posicao_extrato(sc,ultimo_extrato)

#Se o extrato não foi processado no último dia do mês, trazer o último disponível
if df.empty:
    ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
    ultimo_extrato = ultimo_extrato + timedelta(days = -1)
else:
    ultimo_extrato = ultimo_extrato

    
#Gerar a carteira de todas as sc de gestão - portfólios padrão
for i,(index,row) in enumerate(teste.iterrows()):
     if i >0 :
            ultimo_extrato = data
            ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
            ultimo_extrato = ultimo_extrato + timedelta(days = -1)            
            sc = row['new_name']
            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            if df1.empty:
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
            else:
                ultimo_extrato = data
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
                ultimo_extrato

            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            df1['data_ultimo_extrato']=ultimo_extrato          
            df = pd.concat([df,df1])

#Mapeando as informações da base GPS e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')

In [8]:
ultimo_extrato

datetime.date(2024, 10, 31)

In [9]:
sc

'@7tdt2'

In [10]:
#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')

In [11]:
carteira_gps.to_excel('carteira_off.xlsx')

In [12]:
#Levantamento da quantidade de bonds diretos

bonds = df[df['TipoProduto']=='Bond']
bonds = bonds[~bonds['NomeDoProduto'].str.contains('Treasury')]
bonds = bonds[~bonds['NomeDoProduto'].str.contains('US TIPS')]
bonds = bonds[~bonds['NomeDoProduto'].str.contains('Cash Account')]

In [13]:
#Fazer a conciliação de contas
conciliacao = df[['NomeContaCrm','NomeSupercarteiraCrm']].drop_duplicates()
conciliacao=conciliacao['NomeSupercarteiraCrm'].str.lower()
todas_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname']].drop_duplicates()
conciliacao = pd.merge(left=conciliacao, right=todas_sc,left_on='NomeSupercarteiraCrm',right_on='new_name',how='right')
conciliacao = conciliacao[conciliacao['NomeSupercarteiraCrm'].isnull()]
conciliacao.to_excel('conciliacao.xlsx')

In [14]:
#Ajustes nas - futuratmente tirar colunas inúteis
carteira_gps = carteira_gps[['DataPosicao','NomeSupercarteiraCrm','NomeContaCrm','NomeDoProduto','CodigoProduto_x','ISIN','QuantidadeCotas','ValorCota','SaldoNaData','Moeda_x','Classe', 'Subclasse']].rename(columns={'DataPosicao':'Data_Posicao','NomeContaCrm':'Nome_Cliente','NomeDoProduto':'Nome_produto','QuantidadeCotas':'quantidade','ValorCota':'PU','CodigoProduto_x':'CodigoProduto','Moeda_x':'Moeda'})
carteira_gps['Esteira']='ex-Gps'
carteira_gps['Nome_produto']=carteira_gps['Nome_produto'].str.lower().apply(lambda x: arruma_espaco(x))


In [16]:
#Concatenando as carteiras com a custódia - GPS

sc_axys =  crm.axys__sc()
prod_axys = crm.ays_produtos()

import os
caminho = "I:/Shared/Processamento_Conciliacao_Offshore/CARTEIRAS/AXYS/"
lista_arquivos = os.listdir(caminho)
    
lista_datas = []
for arquivo in lista_arquivos:
# descobrir a data desse arquivo
    if ".xlsm" in arquivo:
        datas = os.path.getmtime(f"{caminho}/{arquivo}")
        lista_datas.append((datas, arquivo))    
        lista_datas.sort(reverse=True)
        ultimo_arquivo = lista_datas[0]
        ultimo_arquivo = ultimo_arquivo[1].replace('~$','')
custodias = pd.read_excel(caminho+ultimo_arquivo,sheet_name = 'Axys',header = None)
colunas = custodias.iloc[1]
custodias.columns = colunas
custodias = custodias.drop(index=[0,1])
custodias = custodias[['Portfólio','Security Simbol','Banco']]
colunas = custodias.columns
custodias[colunas] = custodias[colunas].apply(lambda x: x.str.replace(',', '.').astype('str'))
custodias = pd.merge(left =custodias,right=sc_axys,left_on='Portfólio',right_on='new_name',how='left').drop(index=[0,1]).drop(columns={'new_name','Portfólio'})[['new_supercarteiraoriginadoraidname','Security Simbol','Banco']]
custodias = pd.merge(left =custodias,right=prod_axys,left_on='Security Simbol',right_on='new_axyssecuritysymbol',how='left')
carteira_gps = pd.merge(left =carteira_gps,right=custodias,left_on=['NomeSupercarteiraCrm','CodigoProduto'],right_on=['new_supercarteiraoriginadoraidname','new_idsistemaorigem'],how='left').drop(columns={'new_supercarteiraoriginadoraidname','Security Simbol','name','new_idsistemaorigem','new_axyssecuritysymbol'}).rename(columns={'productid':'guidproduto','Banco':'Custódia'})

In [24]:
#Trazer a carteira da Reliance

#Retorna todos os produtos base Reliance
produtos_gestao = rel_off.buscar_fundos_jbfo()

#Traz a recomendação de portfolio por cliente
recomendacao = pd.read_excel('port_rec.xlsx')
recomendacao = recomendacao[['Cliente','Mandato_Recomendado']]

#Puxando a ultima data do mes anterior para calculo do extrato
data_rel_cart = datetime.datetime.date(fdt.hoje() - relativedelta(days=fdt.hoje().day -1)+timedelta(days=-2))

# #Mapeando as informações da base Reliance e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Trazer a última posição disponivel (mes anterior)
carteira_rel = rel_off.carteira_reliance(data_rel_cart)
trazer_data = datetime.datetime.date(carteira_rel['calculado_em'].max())
carteira_rel['calculado_em'] = carteira_rel['calculado_em'].apply(lambda x : datetime.datetime.date(x))
carteira_rel = carteira_rel[carteira_rel['calculado_em'] == trazer_data].rename(columns= {'asset_class':'Subclasse','moeda':'Moeda'}).drop(columns={'Subcategoria'})
carteira_rel = carteira_rel[~(carteira_rel['Nome_produto'].str.contains('FWD'))&(carteira_rel['SaldoNaData']>=0)].drop(columns={'calculado_em'})

#Incluir os daldos na composição da carteira
saldos = rel_off.buscar_saldos()

saldos['ISIN'] = 'caixa'
saldos = saldos[['Data_Posicao', 'Nome_Cliente', 'cd_security', 'ISIN','Nome_produto', 'Classe', 'Subclasse', 'quantidade','PU', 'cotacao', 'SaldoNaData', 'Moeda']]

#Concatenando as carteiras com saldos
carteira_rel = pd.concat([carteira_rel,saldos]).sort_values(by=['Nome_Cliente'])

#Ajuste da Classe para padrão GPS
classes_padrao = pd.read_excel('classificacao.xlsx')[['ID_Produto','cd_asset_class_name']]
carteira_rel = pd.merge(left = carteira_rel, right=classes_padrao,left_on='cd_security', right_on='ID_Produto', how='left')
carteira_rel['Classe'] = carteira_rel['cd_asset_class_name']
carteira_rel = carteira_rel.drop(columns={'ID_Produto','cd_asset_class_name'})

#Remover todos os valores iguais a 0

carteira_rel = carteira_rel.dropna(subset=['SaldoNaData'])

#Ajusta as colunas
carteira_rel['NomeSupercarteiraCrm'] = carteira_rel['Nome_Cliente']
carteira_rel['Esteira'] = 'ex-Reliance'
carteira_rel['CodigoProduto'] = carteira_rel['cd_security'].astype('str') 
carteira_rel = carteira_rel.rename(columns={'moeda':'Moeda'})
carteira_rel = carteira_rel[['Data_Posicao', 'NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto','CodigoProduto', 'ISIN', 'quantidade', 'PU', 'SaldoNaData', 'Moeda','Classe', 'Subclasse', 'Esteira']]


In [37]:
pd.read_excel(file.read(caminho+ultimo_arquivo))

NameError: name 'file' is not defined

In [43]:
#Concatenando as carteiras com a custódia da Reliance

import os
caminho = "I:/Shared/Processamento_Conciliacao_Offshore/CARTEIRAS/SIS INTERNACIONAL/"
lista_arquivos = os.listdir(caminho)
    
lista_datas = []
for arquivo in lista_arquivos:
# descobrir a data desse arquivo
    if ".xls" in arquivo:
        datas = os.path.getmtime(f"{caminho}/{arquivo}")
        lista_datas.append((datas, arquivo))    
        lista_datas.sort(reverse=True)
        ultimo_arquivo = lista_datas[0]
        ultimo_arquivo = ultimo_arquivo[1].replace('~$','')
custodias = pd.read_excel(caminho+ultimo_arquivo,header = None)
colunas = custodias.iloc[0]
custodias.columns = colunas
custodias = custodias.drop(index=[0])
custodias = custodias[['CLIENTE','ATIVO','cd_security','INSTITUIÇÃO']]
colunas = custodias.columns
custodias[colunas] = custodias[colunas].apply(lambda x: x.replace(',', '.').astype('str'))
carteira_rel = pd.merge(left =carteira_rel,right=custodias,left_on=['NomeSupercarteiraCrm','CodigoProduto'],right_on=['CLIENTE','cd_security'],how='left').drop(columns={'CLIENTE','ATIVO','cd_security'}).rename(columns ={'INSTITUIÇÃO':'Custódia'})
carteira_rel['guidproduto'] = 'Rel '+ carteira_rel['CodigoProduto']

In [44]:
carteira_jbfo = pd.concat([carteira_rel,carteira_gps])
carteira_custodia = carteira_jbfo.to_excel('O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Códigos/Carteiras OFF/carteira_jbfo.xlsx')